<a href="https://colab.research.google.com/github/Bahodir011/Bahodir011/blob/main/ml_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/housing_data_08-02-2021.csv')
df.drop(df[df['price'] == 'Договорная'].index, axis = 0, inplace = True)
df.drop(df[df['size'] == 'Площадьземли:1сот'].index, axis = 0, inplace = True)

In [3]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

x_train = train_set.drop("price", axis=1)
y = train_set["price"].copy()



x_num = x_train.drop(["location", 'district'], axis=1)


In [4]:
from sklearn.base import BaseEstimator, TransformerMixin


rooms_ix, size_ix = 2,3

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_size_per_room = True):
        self.add_size_per_room = add_size_per_room
    def fit(self, X, y=None):
        return self # bizni funksiyamiz faqat transformer. estimator emas
    def transform(self, X):
        size_rooms = X[:, size_ix] / X[:, rooms_ix]

        if self.add_size_per_room: # add_bedrooms_per_room ustuni ixtiyoriy bo'ladi
            size_rooms = X[:, size_ix] / X[:, rooms_ix]
            return np.c_[X, size_rooms ]
        else:
            return np.c_[X, size_rooms]

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [6]:
num_pipeline = Pipeline([
          ('imputer', SimpleImputer(strategy='median')),
          ('attribs_adder', CombinedAttributesAdder(add_size_per_room = True)),
          ('std_scaler', StandardScaler())
])

In [7]:
from sklearn.compose import ColumnTransformer

num_attribs = list(x_num)
cat_attribs = ['district', 'location']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

In [8]:
x_prp = full_pipeline.fit_transform(x_train)

In [26]:
x_prp

<5972x1376 sparse matrix of type '<class 'numpy.float64'>'
	with 41804 stored elements in Compressed Sparse Row format>

In [27]:
from sklearn.linear_model import LinearRegression

LR_model = LinearRegression()

In [11]:
LR_model.fit(x_prp, y)

LinearRegression()

In [12]:
test_data = x_train.sample(5)
test_data


,location,district,rooms,size,level,max_levels
2094,"город Ташкент, Мирабадский район, Метро ОЙБЕК",Мирабадский,3,75,9,9
3895,"город Ташкент, Сергелийский район, Янгихаётски...",Сергелийский,2,59.80,16,16
7338,"город Ташкент, Яшнободский район, 1-й проезд А...",Яшнободский,1,24,1,4
7127,"город Ташкент, Шайхантахурский район, Чорсу",Шайхантахурский,3,72,8,9
6486,"город Ташкент, Учтепинский район, Чиланзар 11-...",Учтепинский,4,80,3,4


In [14]:
test_l = y.loc[test_data.index]
test_l

2094    80000
3895    27288
7338    19000
7127    69000
6486    52500
Name: price, dtype: object

In [15]:
test_data_prepared = full_pipeline.transform(test_data)
test_data_prepared

<5x1376 sparse matrix of type '<class 'numpy.float64'>'
	with 35 stored elements in Compressed Sparse Row format>

In [16]:
predicted_data = LR_model.predict(test_data_prepared)
predicted_data

array([79974.77259688, 27249.54392343,  9532.72720398, 77068.40408921,
       71397.48263008])

In [17]:
pd.DataFrame({'Prognoz':predicted_data, 'Real baxosi': test_l})

,Prognoz,Real baxosi
2094,79974.772597,80000
3895,27249.543923,27288
7338,9532.727204,19000
7127,77068.404089,69000
6486,71397.482630,52500


In [18]:
x_test = test_set.drop('price',  axis =1)
x_test

,location,district,rooms,size,level,max_levels
2301,"город Ташкент, Яшнободский район, Паркентский ...",Яшнободский,3,86,4,9
6126,"город Ташкент, Учтепинский район, Чиланзар 22-...",Учтепинский,3,70,2,5
2149,"город Ташкент, Яккасарайский район, Кушбеги",Яккасарайский,3,90,5,5
5594,"город Ташкент, Чиланзарский район, Чиланзар-16",Чиланзарский,1,36,2,4
6566,"город Ташкент, Учтепинский район, Кухна-Чупоно...",Учтепинский,4,100,1,4
...,...,...,...,...,...,...
6593,"город Ташкент, Шайхантахурский район, Караташ",Шайхантахурский,3,80,6,9
4089,"город Ташкент, Шайхантахурский район, Актепа",Шайхантахурский,6,500,2,2
441,"город Ташкент, Чиланзарский район, Чиланзар-6",Чиланзарский,1,36,2,5
5639,"город Ташкент, Шайхантахурский район, Урда",Шайхантахурский,3,70,6,7


In [19]:
test_set

,location,district,rooms,size,level,max_levels,price
2301,"город Ташкент, Яшнободский район, Паркентский ...",Яшнободский,3,86,4,9,89000
6126,"город Ташкент, Учтепинский район, Чиланзар 22-...",Учтепинский,3,70,2,5,43500
2149,"город Ташкент, Яккасарайский район, Кушбеги",Яккасарайский,3,90,5,5,45000
5594,"город Ташкент, Чиланзарский район, Чиланзар-16",Чиланзарский,1,36,2,4,24700
6566,"город Ташкент, Учтепинский район, Кухна-Чупоно...",Учтепинский,4,100,1,4,87000
...,...,...,...,...,...,...,...
6593,"город Ташкент, Шайхантахурский район, Караташ",Шайхантахурский,3,80,6,9,48000
4089,"город Ташкент, Шайхантахурский район, Актепа",Шайхантахурский,6,500,2,2,1800
441,"город Ташкент, Чиланзарский район, Чиланзар-6",Чиланзарский,1,36,2,5,28500
5639,"город Ташкент, Шайхантахурский район, Урда",Шайхантахурский,3,70,6,7,60000


In [20]:
x_test = test_set.drop('price', axis =1)
x_test

,location,district,rooms,size,level,max_levels
2301,"город Ташкент, Яшнободский район, Паркентский ...",Яшнободский,3,86,4,9
6126,"город Ташкент, Учтепинский район, Чиланзар 22-...",Учтепинский,3,70,2,5
2149,"город Ташкент, Яккасарайский район, Кушбеги",Яккасарайский,3,90,5,5
5594,"город Ташкент, Чиланзарский район, Чиланзар-16",Чиланзарский,1,36,2,4
6566,"город Ташкент, Учтепинский район, Кухна-Чупоно...",Учтепинский,4,100,1,4
...,...,...,...,...,...,...
6593,"город Ташкент, Шайхантахурский район, Караташ",Шайхантахурский,3,80,6,9
4089,"город Ташкент, Шайхантахурский район, Актепа",Шайхантахурский,6,500,2,2
441,"город Ташкент, Чиланзарский район, Чиланзар-6",Чиланзарский,1,36,2,5
5639,"город Ташкент, Шайхантахурский район, Урда",Шайхантахурский,3,70,6,7


In [21]:
y_test = test_set['price'].copy()
y_test

2301    89000
6126    43500
2149    45000
5594    24700
6566    87000
        ...  
6593    48000
4089     1800
441     28500
5639    60000
2538    42000
Name: price, Length: 1493, dtype: object

In [22]:
x_test_pr = full_pipeline.fit_transform(x_test)



In [31]:
y_pr = LR_model.predict(x_test_pr)

NotFittedError: ignored

In [138]:
x_test_pr.shape

(1493, 612)